<a href="https://colab.research.google.com/github/WeizmannML/course2020/blob/master/Tutorial2/Pytorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <font color=blue>Pytorch allows us to use automatic differentiation & use the power of GPU </font>
#### All the code snippets in the following example are taken from https://pytorch.org/tutorials/beginner/pytorch_with_examples.html 
#### Other libraries like tensorflow or JAX allows us to do the same. 

In [0]:
from __future__ import print_function
import torch

In [2]:
torch.__version__

'1.5.0+cu101'

In [0]:
import os, sys
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [0]:
# ----- for automatic detection of CPU vs GPU device ---------- #
cuda_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu' )

#### The basic working objects are torch.Tensors 

In [0]:
a = torch.tensor([[1., -1.], [1., -1.]])

In [0]:
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))

In [7]:
type(a), type(b)

(torch.Tensor, torch.Tensor)

In [8]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [9]:
x = x.new_ones(5, 3, dtype=torch.double, device=cuda_device)      # new_* methods take in sizes
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='cuda:0', dtype=torch.float64)


In [10]:
x.device ### equivalently can be done as x = x.to(cuda_device)

device(type='cuda', index=0)

In [11]:
x = torch.randn_like(x, dtype=torch.float64)    # override dtype!
print(x) 

tensor([[-2.2291,  0.2696, -0.4319],
        [-0.4727,  0.9656,  0.8269],
        [ 0.5744,  1.3900, -1.1216],
        [-0.8299, -2.7469,  0.3398],
        [-0.9268,  0.4171, -1.8334]], device='cuda:0', dtype=torch.float64)


In [12]:
print(x.shape), print(x.size())

torch.Size([5, 3])
torch.Size([5, 3])


(None, None)

#### Conversion to numpy arrays

In [0]:
y = x.cpu().numpy()

In [14]:
y

array([[-2.2291205 ,  0.26962344, -0.43190726],
       [-0.47273675,  0.9656231 ,  0.8269411 ],
       [ 0.574432  ,  1.38996071, -1.12160237],
       [-0.82992153, -2.74685504,  0.33983331],
       [-0.92684507,  0.41711241, -1.83343549]])

In [15]:
y.dtype

dtype('float64')

## The automatic differentiation of tensors through autograd

In [16]:
x = x.new_ones(5, 3, dtype=torch.double, device=cuda_device)

x = torch.randn_like(x, dtype=torch.double, device=cuda_device, requires_grad=True)
print(x)

tensor([[-0.4090, -0.5666, -0.8329],
        [ 1.6109,  2.1927,  0.8926],
        [ 0.5376,  0.9540,  1.8133],
        [-1.7818,  1.2109,  1.7909],
        [ 1.1760, -1.1186, -0.0306]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)


In [17]:
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]), dtype=torch.double, device=cuda_device, requires_grad=True)
print(b)

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)


In [18]:
y = x + 2
print(y)

tensor([[1.5910, 1.4334, 1.1671],
        [3.6109, 4.1927, 2.8926],
        [2.5376, 2.9540, 3.8133],
        [0.2182, 3.2109, 3.7909],
        [3.1760, 0.8814, 1.9694]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)


In [19]:
y.grad_fn

In [20]:
## -- assigning a required_grad criteria -- #
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [21]:
print(a), print(b)

tensor([[  1.2504,  -9.7092],
        [ 65.1794, -78.6856]], requires_grad=True)
tensor(10535.6172, grad_fn=<SumBackward0>)


(None, None)

In [22]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 664.3240, -776.3787,  827.5244], grad_fn=<MulBackward0>)


In [23]:
x

tensor([ 0.6488, -0.7582,  0.8081], requires_grad=True)

In [24]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


### Doing a backpropagation in the numpy way with torch tensors

In [25]:

dtype = torch.float
device = cuda_device
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0) # -- the clamp filters out < 0 values
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 50 == 49:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h) # -- we had x.T for numpy arrays

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

49 11629.7216796875
99 299.8217468261719
149 13.28006362915039
199 0.7252171039581299
249 0.04478435218334198
299 0.0032201511785387993
349 0.0004235576489008963
399 0.00011634056136244908
449 4.920796709484421e-05
499 2.7687812689691782e-05


## Using the autograd function of pytorch to compute the derivatives & do backpropagation

In [26]:
# -*- coding: utf-8 -*-
dtype = torch.float
device = cuda_device
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2) # --- the forward pass through two layers is happening here.

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

49 13565.404296875
99 457.6578063964844
149 26.036725997924805
199 1.8840091228485107
249 0.15825319290161133
299 0.014900850132107735
349 0.0017812233418226242
399 0.00036286894464865327
449 0.0001212550705531612
499 5.6581091484986246e-05


### The PyTorch nn Modeule, contains the basic NN layers

In [28]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in, device=cuda_device)
y = torch.randn(N, D_out, device=cuda_device)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

model.to(cuda_device)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

49 30.213838577270508
99 1.9144725799560547
149 0.20878511667251587
199 0.02865973673760891
249 0.004503307398408651
299 0.0007870843401178718
349 0.00014831690350547433
399 2.951384158222936e-05
449 6.119999852671754e-06
499 1.310145535171614e-06


### Updating the weights through PyTorch optim 

In [29]:

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in, device=cuda_device)
y = torch.randn(N, D_out, device=cuda_device)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

model.to(cuda_device)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

49 190.23922729492188
99 44.95762634277344
149 7.090780735015869
199 0.6727340221405029
249 0.04889321327209473
299 0.004506479017436504
349 0.0005203515174798667
399 5.274931027088314e-05
449 4.16265856983955e-06
499 2.486897869857785e-07
